# 환경설정

In [ ]:
import re
import json

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # 처음 실행 시 브라우저 인증
drive = GoogleDrive(gauth)

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
googlr_drive_rt_id = os.getenv("GOOGLE_DRIVE_RT_ID")
google_dirve_vtt_id = os.getenv("GOOGLE_DRIVE_VTT_ID")
google_dirve_txt_id = os.getenv("GOOGLE_DRIVE_TXT_ID")
google_dirve_md_id = os.getenv("GOOGLE_DRIVE_MD_ID")

In [ ]:
def upload_to_drive(filename: str, data, google_drive_id: str):
    """ 데이터 구글 드라이브에 저장 """
    file = drive.CreateFile({
        'title': filename,
        'parents': [{'id': google_drive_id}]
    })
    file.SetContentString(data)
    file.Upload()

# 데이터 수집

- 유튜브 플레이 리스트 제목 : 조회수 100만 이상의 즉문즉설 베스트
- 유뷰브 플레이 리스트 ID : `PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP`

In [ ]:
# 로컬 저장
# ! yt-dlp --write-auto-sub --sub-lang ko --skip-download -o "data/%(title)s.%(ext)s" "https://www.youtube.com/playlist?list=PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP"

In [ ]:
import yt_dlp
import requests

def get_playlist_entries(playlist_url):
    """ 유튜브 플레이리스트 URL 데이터 반환"""
    ydl_opts = {'quiet': True, 'extract_flat': True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(playlist_url, download=False)
        return info['entries']
    
def get_subtitle_text(info_dict):
    """ 유튜브 자막 데이터 반환 """
    # 자막 가져오기
    subtitles = info_dict.get('subtitles') or info_dict.get('automatic_captions')
    
    if not subtitles or 'ko' not in subtitles:
        return None, None
    
    # 자막 가져오기
    subtitle_url = subtitles['ko'][0]['url']
    response = requests.get(subtitle_url)
    response.encoding = 'utf-8'
    return info_dict['title'], response.text

In [ ]:
playlist_id = "PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP"
playlist_url = f"https://www.youtube.com/playlist?list={playlist_id}"
entries = get_playlist_entries(playlist_url)

ydl_opts = {
    'writesubtitles': True,
    'skip_download': True,
    'subtitleslangs': ['ko'],
    'writeautomaticsub': True,
    'quiet': True,
    'outtmpl': '-',  # 실제 저장은 안함
}
ydl = yt_dlp.YoutubeDL(ydl_opts)

meta_data = {}
for entry in entries:
    video_url = f"https://www.youtube.com/watch?v={entry['id']}"
    info_dict = ydl.extract_info(video_url, download=False)
    title, text = get_subtitle_text(info_dict)
    if title and text:
        file_name = f"{title}.ko.vtt"
        upload_to_drive(file_name, text, google_dirve_vtt_id)
        meta_data[video_url] = {
            'title': title,
            'tags': info_dict.get('tags'),
            'view_count': info_dict.get('view_count'),
            'duration': info_dict.get('duration'),
            'like_count': info_dict.get('like_count'),
            'channel': info_dict.get('channel'),
            'upload_date': info_dict.get('upload_date'),
        }
    else:
        print(f"❌ 자막 없음: {video_url}")

upload_to_drive('yt_vtt_meta.json', meta_data, google_dirve_vtt_id)

# 데이터 가공

### 1차 가공 (vtt → txt)

In [ ]:
def clean_content(title:str, content:dict) -> str:
    """ 자막 데이터 1차 가공 """
    if "몰아보기" in title:
        print(title, '..Skip..')
        print('==='*20)
        return None
    
    if "events" not in content:
        print(title, '..Empty..')
        print('==='*20)
        return None

    segs = []
    for event in content['events']:
        if "segs" not in event:
            continue

        segs += event["segs"]
    
    cleaned_content = ' '.join([seg["utf8"] for seg in segs])
    cleaned_content = cleaned_content.replace('[박수]', ' ')
    cleaned_content = cleaned_content.replace('[웃음]', ' ')
    cleaned_content = cleaned_content.replace('[음악]', ' ')
    cleaned_content = cleaned_content.replace('(청중 웃음)', ' ')
    cleaned_content = cleaned_content.replace('(청중 박수)', ' ')
    cleaned_content = re.sub(r'\s+', ' ', cleaned_content).strip()

    # 가공 데이터 구글 드라이브에 저장
    upload_to_drive(title, cleaned_content, google_dirve_txt_id)

    print(title)
    print('---'*20)
    print(f'{cleaned_content[:10]}...[{len(cleaned_content)}]', )
    print('==='*20)
    return cleaned_content

In [ ]:
# .vtt 파일만 검색
file_list = drive.ListFile({
    'q': f"'{google_dirve_vtt_id}' in parents and title contains '.vtt' and trashed=false"
}).GetList()
len(file_list)

In [ ]:
# 자막 데이터 로드
contents = {}
for file in file_list[40:50]:
    title = re.sub(".vtt", ".txt", file['title'])
    content = json.loads(file.GetContentString())
    content = clean_content(title, content) # 자막 데이터 1차 가공
    contents[title] = content

### 2차 가공 (txt → md)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.1
)

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain_core.output_parsers import StrOutputParser

# 자막 요약 체인 구성
system_template = """당신은 법륜스님의 즉문즉설 자막을 요약하는 AI입니다. 
당신의 작업은 다음과 같습니다:

1. 자막 전체를 읽고 누가 말했는지 판단합니다. 입력으로 주어지는 자막은 줄글이며, 화자가 명시되지 않은 경우도 있습니다. 자막에는 가끔 (스님), (질문자) 같은 표기가 포함되어 있는데, 이 표기는 다음 대사의 화자를 의미합니다. 예: (질문자) 네 → 질문자가 "네"라고 말한 것  / (스님) 괜찮아 → 스님이 "괜찮아"라고 말한 것
2. 대화 흐름을 유지한 채로 "스님:"과 "질문자:" 형식으로 대사를 나눕니다.  
3. 말의 핵심 내용만 남겨 **간결하게 요약**하되, 대화의 흐름과 말투는 유지합니다. 말투와 분위기는 자연스럽게 유지해 주세요. 경전 이름, 일화, 농담, 강조가 있으면 그대로 살려 주세요.
4. 생략 없이, 말한 사람과 순서를 정확히 반영해 주세요.

결과는 아래 형식으로 출력해 주세요:

질문자: (요약된 질문)  
스님: (요약된 답변)  
질문자: (요약된 중간 반응 또는 후속 질문)  
스님: (요약된 답변)
"""
system_message = SystemMessagePromptTemplate.from_template(template=system_template)

human_template = """다음은 즉문즉설 자막입니다. 이 내용을 대화 형식을 유지하면서 요약해 주세요.

[자막 원문]
{content}

[자막 요약]
"""
human_message = HumanMessagePromptTemplate.from_template(template=human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
chain = chat_prompt | llm | StrOutputParser()

In [ ]:
# .txt 파일만 검색
txt_file_list = drive.ListFile({
    'q': f"'{google_dirve_txt_id}' in parents and title contains '.txt' and trashed=false"
}).GetList()
len(txt_file_list)

In [ ]:
for file in txt_file_list:
    title = re.sub(".txt", ".md", file['title'])
    content = file.GetContentString()               # 자막 데이터 로드
    content = chain.invoke({"content": content})    # 자막 데이터 2차 가공

    # 가공 데이터 구글 드라이브에 저장
    upload_to_drive(title, content, google_dirve_md_id)

    print(title)
    print('---'*10)
    print(content[:100])
    print('==='*10)